In [ ]:
# import libraries
import numpy as np
import sys
import psi4
from helper_PFCI import PFHamiltonianGenerator
np.set_printoptions(threshold=sys.maxsize)
psi4.core.set_output_file('output.dat', False)
import json
import git


In [ ]:
# get commit number for github repository
repo = git.Repo(search_parent_directories=True)
sha = repo.head.object.hexsha
print(sha)

In [ ]:
# setup basic arguments for qed-ci calculation
mol_str = """
Li
H 1 1.8
symmetry c1
"""

options_dict = {
    "basis": "sto-3g",
    "scf_type": "pk",
    "e_convergence": 1e-10,
    "d_convergence": 1e-10,
}

cavity_dict = {
    'omega_value' : 0.12086,
    'lambda_vector' : np.array([0, 0, 0.05]),
    'ci_level' : 'fci',
    'full_diagonalization' : True,
    'number_of_photons' : 10, 
}




In [ ]:
# set up base dictionary - some of this will be updated with each calculation
dictionary = {
  "repo" : {
      
      "repository_url" : "https://github.com/mapol-chem/qed-ci",
      "branch" : "photon_number_basis",
      "commit" : sha
  },
    
  "molecule" : {
    "molecule_name": "LiH",
    "geometry": [],
    "symbols": [
      "Li",
      "H"
    ],
    "z-matrix" : mol_str,
  },
    "driver": "energy",
    "model" : {
        "method" : "qed-fci",
        "orbital_basis"  : "sto-3g",
        "photon_basis" : "number_basis",
        "number_of_photon_states" : cavity_dict["number_of_photons"],
        "lambda" : [
            cavity_dict["lambda_vector"][0],
            cavity_dict["lambda_vector"][1],
            cavity_dict["lambda_vector"][2]
        ],
        "omega" : cavity_dict["omega_value"]  
    },
    
    "return_result" : [
        
    ],
    
    "cavity_free_rhf_energy" : 0.,
    "qed_rhf_energy" : 0.,
    "cavity_free_fci_ground_state_energy" : 0.,
}

# function to generate file names based on system details
def generate_file_name(dic, r):
    string_r = "{:.3f}".format(r)
    file_name = dic["molecule"]["molecule_name"] + "_"
    file_name += "r_" + string_r + "_"
    file_name += str(dictionary["model"]["method"]) + "_"
    file_name += str(dictionary["model"]["orbital_basis"]) + "_"
    file_name += str(dictionary["model"]["photon_basis"]) + "_"
    file_name += str(dictionary["model"]["number_of_photon_states"]) + "_"
    file_name += "lambda_" + str(dictionary["model"]["lambda"][0]) 
    file_name += "_" + str(dictionary["model"]["lambda"][1]) 
    file_name += "_" + str(dictionary["model"]["lambda"][2]) + "_"
    file_name += "omega_" + str(dictionary["model"]["omega"]) + ".json"
    return file_name

In [ ]:
# print a test dictionary and filename
print(dictionary)
print(generate_file_name(dictionary, 1.833443))

In [ ]:
# scan through bond-stretch and write .json file for each geometry
mol_tmpl = """
Li
H 1 **R**
symmetry c1
"""

mol_str = """
Li
H 1 1.5
symmetry c1
"""

N_R = 50
N_el = 50
r_array = np.linspace(1.4, 2.2, N_R)

E_array = np.zeros((N_R, N_el))

r_idx = 0
for r in r_array:
    file_name = generate_file_name(dictionary, r) 
    mol_str = mol_tmpl.replace("**R**", str(r))
    mol = psi4.geometry(mol_str)
    #psi4.set_options({'restricted_docc': [3],'active': [4],'num_roots':2})
    scf_e, wfn = psi4.energy('SCF/sto-3g', return_wfn=True)
    fci_energy = psi4.energy('fci',ref_wfn=wfn)    
    LiH = PFHamiltonianGenerator(mol_str, options_dict, cavity_dict)
    
    dictionary["return_result"] = [LiH.CIeigs[0], LiH.CIeigs[1], LiH.CIeigs[2], LiH.CIeigs[3], LiH.CIeigs[4], LiH.CIeigs[5], LiH.CIeigs[6]]
    dictionary["cavity_free_rhf_energy"] = scf_e
    dictionary["cavity_free_fci_ground_state_energy"] = fci_energy
    geometry = np.asarray(mol.geometry())
    geometry_list = geometry.tolist()
    dictionary["molecule"]["geometry"] = geometry_list
    E_array[r_idx, :] = LiH.CIeigs[:50]
    r_idx += 1
    print(dictionary)
    json_object = json.dumps(dictionary, indent=4)
    with open(file_name, "w") as outfile:
        outfile.write(json_object)

